# **Homework:** 02-Experiment Tracking

In [1]:
# Loading utils libraries
import os

# Loading MLFlow libraries
import mlflow

In [2]:
# Q1. What is the version of the MLflow library?
print("MLflow version:", mlflow.__version__)    

MLflow version: 2.13.0


For Q2 i need to run the next command:

```python preprocess_data.py --raw_data_path green_taxi_data --dest_path ./output```

In [3]:
# Q2. Total output files in the directory
files = os.listdir('output')
print("The total number of files in the output directory is:", len(files), ", whit this extensions:", set([os.path.splitext(file)[1] for file in files]) )

The total number of files in the output directory is: 4 , whit this extensions: {'.pkl'}


In [4]:
# Q3. Train a model with autolog
import os
import pickle

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import root_mean_squared_error

In [10]:
# mlflow configuration
mlflow.set_tracking_uri("02-experiment-traking/mlflow.db")
mlflow.set_experiment("02-experiment-tracking-autolog")

2024/05/29 12:34:12 INFO mlflow.tracking.fluent: Experiment with name '02-experiment-tracking-autolog' does not exist. Creating a new experiment.


<Experiment: artifact_location='/home/mateo/Escritorio/Proyectos/mlops-zoomcamp/02-experiment-traking/02-experiment-traking/mlflow.db/492000004112917738', creation_time=1717004053397, experiment_id='492000004112917738', last_update_time=1717004053397, lifecycle_stage='active', name='02-experiment-tracking-autolog', tags={}>

In [6]:
def load_pickle(filename: str):
    with open(filename, "rb") as f_in:
        return pickle.load(f_in)

In [11]:
data_path = "./output"

X_train, y_train = load_pickle(os.path.join(data_path, "train.pkl"))
X_val, y_val = load_pickle(os.path.join(data_path, "val.pkl"))

mlflow.autolog()

with mlflow.start_run():
    rf = RandomForestRegressor(max_depth=10, random_state=0)
    rf.fit(X_train, y_train)
    y_pred = rf.predict(X_val)

    rmse = root_mean_squared_error(y_val, y_pred)
    print("Random Forest RMSE:", rmse)

2024/05/29 12:36:39 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2024/05/29 12:36:40 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'
